In [2]:
import streamlit_functions as sf
import importlib as imp
import geopandas as gpd
import matplotlib.pyplot as plt
import functions as f
# debugging
import importlib as imp
imp.reload(f)

# graph
import networkx as nx

<module 'functions' from 'd:\\ITU\\Geospatial Data Science\\PROJECT\\functions.py'>

In [3]:
bikelane_all = gpd.read_parquet('dataset/raw_unprocessed/bikelane_dk_crs_25832.parquet')

In [44]:
G_edges = nx.Graph()

for row in bikelane_all.itertuples():
    # Access the MultiIndex values (u, v, key) from the Index
    u, v, _ = row.Index

    # Add nodes
    G_edges.add_node(u)
    G_edges.add_node(v)

    # Add edge
    G_edges.add_edge(u, v)


In [45]:
# Write to GraphML
nx.write_graphml(G_edges, 'graph.graphml')

In [6]:
coordinates1 = [(669963.1858043118, 6156595.619894134), (666199.299537813, 6155152.20113962)] # somewhere
coordinates2 = [(719420.4753819183, 6177679.685542446), (720676.1469959107, 6176793.722376416)] # in Copenhagen
coordinates3 = [(716235.4790146917, 6208167.005935997), (693675.6526506029, 6101757.298274318)] # big distance

### Without indexing the below code is super slow since it loops through the whole database to find the closest bikelane

In [42]:
import geopandas as gpd
import networkx as nx
from shapely.geometry import Point, LineString
from shapely.ops import nearest_points

# Assuming 'bikelane_all' is your GeoDataFrame of bike lanes
# and 'coordinates2' are your start and end points

# Convert points to Shapely Points
start_point = Point(coordinates2[0])
end_point = Point(coordinates2[1])

# Function to find the closest LineString to a point
print("findd closest linestring start")
def find_closest_linestring(point, gdf):
    min_dist = float('inf')
    closest_line = None
    for line in gdf.geometry:
        dist = point.distance(line)
        if dist < min_dist:
            min_dist = dist
            closest_line = line
    return closest_line

# Find closest LineStrings to start and end points
closest_line_start = find_closest_linestring(start_point, bikelane_all)
closest_line_end = find_closest_linestring(end_point, bikelane_all)
print("closest_lines are:")
print(closest_line_start)
print(closest_line_end)

print("findd closest linestring end")
# Find closest nodes to start and end points
start_node, _ = nearest_points(start_point, closest_line_start)
end_node, _ = nearest_points(end_point, closest_line_end)
print("nearest_points end")
# Convert bike lanes network to a graph

"""G = nx.Graph()
for index, row in bikelane_all.iterrows():
    p1, p2 = row['geometry'].boundary
    G.add_edge(p1.coords[0], p2.coords[0], weight=row['geometry'].length)

# Find shortest path
path = nx.shortest_path(G, source=start_node.coords[0], target=end_node.coords[0], weight='weight')

# Convert path to LineString
path_linestring = LineString(path)

# Convert LineString to GeoDataFrame for visualization
path_gdf = gpd.GeoDataFrame(geometry=[path_linestring], crs=bikelane_all.crs)"""

findd closest linestring start
closest_lines are:
LINESTRING (719397.7148862425 6177829.284857129, 719405.0846778937 6177801.8819245845, 719421.6396323182 6177740.2992502665, 719423.9498579694 6177731.696480225, 719435.0308848498 6177689.015741999, 719444.0791641185 6177654.18242757, 719440.0535103269 6177646.34183948)
LINESTRING (720712.8564264507 6176820.9603884565, 720641.195113393 6176788.851636762)
findd closest linestring end
nearest_points end


"G = nx.Graph()\nfor index, row in bikelane_all.iterrows():\n    p1, p2 = row['geometry'].boundary\n    G.add_edge(p1.coords[0], p2.coords[0], weight=row['geometry'].length)\n\n# Find shortest path\npath = nx.shortest_path(G, source=start_node.coords[0], target=end_node.coords[0], weight='weight')\n\n# Convert path to LineString\npath_linestring = LineString(path)\n\n# Convert LineString to GeoDataFrame for visualization\npath_gdf = gpd.GeoDataFrame(geometry=[path_linestring], crs=bikelane_all.crs)"